## ПЗ 9

In [ ]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import tensorflow as tf
import os

In [ ]:
path_to_file = 'evgenyi_onegin.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

text = text.split('\n\n')

In [ ]:
text[6]

'                        "Мой дядя самых честных правил,\n                        Когда не в шутку занемог,\n                        Он уважать себя заставил\n                        И лучше выдумать не мог.\n                        Его пример другим наука;\n                        Но, боже мой, какая скука\n                        С больным сидеть и день и ночь,\n                        Не отходя ни шагу прочь!\n                        Какое низкое коварство\n                        Полуживого забавлять,\n                        Ему подушки поправлять,\n                        Печально подносить лекарство,\n                        Вздыхать и думать про себя:\n                        Когда же черт возьмет тебя!"'

In [ ]:
len(text)

782

In [ ]:
text_only = []
for strofa in text:
    if len(strofa) < 350:
        continue
    else:
        text_only.append(strofa)
len(text_only)

376

In [ ]:
text_only[155]

'                        Кого ж любить? Кому же верить?\n                        Кто не изменит нам один?\n                        Кто все дела, все речи мерит\n                        Услужливо на наш аршин?\n                        Кто клеветы про нас не сеет?\n                        Кто нас заботливо лелеет?\n                        Кому порок наш не беда?\n                        Кто не наскучит никогда?\n                        Призрака суетный искатель,\n                        Трудов напрасно не губя,\n                        Любите самого себя,\n                        Достопочтенный мой читатель!\n                        Предмет достойный: ничего\n                        Любезней, верно, нет его.'

In [ ]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "Content"})
data

,Content
0,Не мысля гордый свет з...
1,"""Мой дядя самых честны..."
2,Так думал молодой пове...
3,Служив отлично благоро...
4,Когда же юности мятежн...
...,...
371,А счастье было так воз...
372,Она ушла. Стоит Евгени...
373,"Кто б ни был ты, о мой..."
374,"Прости ж и ты, мой спу..."


In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()




def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)    
    txt = re.sub("\n", " \n", txt)    
    return txt



def preprocess_text(txt, morph = False):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\n", "zzz", txt)
    new_txt =[]
    for word in txt.split():
        if word == "zzz":
            word = " \n"
            
        else:
            if morph:
                word = morpher.parse(word)[0].normal_form
            else:
                pass
        new_txt.append(word)

    return new_txt


data['Content_splited'] = data['Content'].apply(exclude_punctuation)


data['Content_splited_morph'] = data['Content_splited'].apply(preprocess_text, morph = True)
data['Content_splited'] = data['Content_splited'].apply(preprocess_text, morph = False)

data

,Content,Content_splited,Content_splited_morph
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима...","[не, мыслить, гордый, свет, забавить, \n, вни..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда...","[мой, дядя, самый, честной, правило, \n, когд..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы...","[так, думать, молодой, повеса, \n, лететь, в,..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи...","[служивый, отлично, благородно, \n, долг, жит..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг...","[когда, же, юность, мятежный, \n, прийти, евг..."
...,...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл...","[а, счастие, быть, так, возможно, \n, так, бл..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г...","[она, уйти, стоить, евгений, \n, как, будто, ..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д...","[кто, б, ни, быть, ты, о, мой, читатель, \n, ..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n...","[простить, ж, и, ты, мой, спутник, странный, ..."


In [ ]:
def get_w2i_i2w(column_data):
    
    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))

    word2idx = {u:i for i, u in enumerate(vocab)}
    idx2word = np.array(vocab)
    
    print(len(vocab), len(word2idx), len(idx2word))
    return word2idx, idx2word
    

Дальше можно работать с одним из двух вариантов.   
Первый это с текстом __"как есть"__.    
Второй с приведенным к __нормальной форме__.

Для примера далее работаем с текстом __"как есть"__.

In [ ]:
word2idx, idx2word = get_w2i_i2w(data['Content_splited'])        
data['int_Content_splited'] = data['Content_splited'].apply(lambda x: [word2idx[c] for c in x])            

8427 8427 8427


In [ ]:
data

,Content,Content_splited,Content_splited_morph,int_Content_splited
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима...","[не, мыслить, гордый, свет, забавить, \n, вни...","[3817, 3634, 1358, 6327, 2071, 0, 878, 1844, 9..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда...","[мой, дядя, самый, честной, правило, \n, когд...","[3487, 1912, 6292, 8129, 5394, 0, 2788, 3817, ..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы...","[так, думать, молодой, повеса, \n, лететь, в,...","[7249, 1867, 3505, 4930, 0, 3102, 565, 5872, 3..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи...","[служивый, отлично, благородно, \n, долг, жит...","[6660, 4535, 325, 0, 1720, 2041, 1919, 4512, 0..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг...","[когда, же, юность, мятежный, \n, прийти, евг...","[2788, 1980, 8363, 3638, 0, 5651, 1915, 5222, ..."
...,...,...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл...","[а, счастие, быть, так, возможно, \n, так, бл...","[98, 7202, 550, 7249, 924, 0, 7249, 380, 4072,..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г...","[она, уйти, стоить, евгений, \n, как, будто, ...","[4371, 7870, 7006, 1914, 0, 2637, 515, 1462, 5..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д...","[кто, б, ни, быть, ты, о, мой, читатель, \n, ...","[2960, 160, 4037, 546, 7548, 4141, 3487, 8153,..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n...","[простить, ж, и, ты, мой, спутник, странный, ...","[5751, 1951, 2446, 7548, 3487, 6916, 7049, 0, ..."


In [ ]:
def get_all_int(column_data):
    
    int_dump = column_data.values
    all_txt_as_int = []

    for sublist in int_dump:
        for item in sublist:
            all_txt_as_int.append(item)
    all_txt_as_int = np.array(all_txt_as_int)    
    
    return all_txt_as_int

all_txt_as_int = get_all_int(data['int_Content_splited'])

In [ ]:
seq_length = 45
examples_per_epoch = len(all_txt_as_int)//(seq_length+1)

word_dataset = tf.data.Dataset.from_tensor_slices(all_txt_as_int)



for i in word_dataset.take(15):
    print(idx2word[i.numpy()])

не
мысля
гордый
свет
забавить
 

вниманье
дружбы
возлюбя
 

хотел
бы
я
тебе
представить


In [ ]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'не мысля гордый свет забавить  \n вниманье дружбы возлюбя  \n хотел бы я тебе представить  \n залог достойнее тебя  \n достойнее души прекрасной  \n святой исполненной мечты  \n поэзии живой и ясной  \n высоких дум и простоты  \n но так и быть рукой пристрастной  \n'
Target data: 'мысля гордый свет забавить  \n вниманье дружбы возлюбя  \n хотел бы я тебе представить  \n залог достойнее тебя  \n достойнее души прекрасной  \n святой исполненной мечты  \n поэзии живой и ясной  \n высоких дум и простоты  \n но так и быть рукой пристрастной  \n прими'


In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 45), (64, 45)), types: (tf.int64, tf.int64)>

In [ ]:
vocab_size = len(idx2word)

embedding_dim = 256

rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [ ]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 8427)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 45, 8427) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (64, None, 256)      2157312     input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (64, None, 1024)     3938304     embedding[0][0]                  
__________________________________________________________________________________________________
concatenate (Concatenate)       (64, None, 1280)     0           embedding[0][0]                  
                                                                 gru[0][0]                    

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 45, 8427)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.039167


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [ ]:
EPOCHS = 150

In [ ]:
for i in range(EPOCHS):
    model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])
    model.reset_states()

9/9 [==============================] - 4s 409ms/step - loss: 0.0643


9/9 [==============================] - 4s 408ms/step - loss: 0.0534


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_1'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 8427)


In [ ]:
def generate_text(model, start_string):
    
    start_string = exclude_punctuation(start_string)
    start_string_asis = preprocess_text(start_string, morph = False)

    num_generate = 30

    input_eval = [word2idx[s] for s in start_string_asis]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.3

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"дядя "))

дядя скоро все перевелось  
 корсет альбом княжну алину  
 стишков чувствительных тетрадь  
 она забыла стала звать  
 акулькой прежнюю селину  
 и обновила наконец  
 на вате шлафор и


In [ ]:
start_string = "Мой дядя  "
print(generate_text(model, start_string=start_string))

Мой дядя  утра навсегда читать б еще ты был проказник  
 и тот же шпиц и тот же муж  
 а он все клуба член исправный  
 все так же смирен так
